In [ ]:
# A Playground for prompt engineering

## Set-up

In [ ]:
%pip install python-dotenv
%pip install openai
%pip install --upgrade tiktoken
%pip install PyJSONViewer

In [1]:
import os
from dotenv import load_dotenv, find_dotenv # load keys from .env file
import openai # use OpenAI API
import tiktoken # use OpenAI token counter API
import wolframalpha # use Wolfram Alpha API
import random

import json
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

encoding = tiktoken.get_encoding("cl100k_base")

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    print(">>>>")
    print(str(response.choices[0].message["content"]))
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [11]:

myPrompt = "while a language model is suitable for natural language tasks, it is not suitable for tasks that demand analytical thinking with a structured approach because ..."
print(get_completion(myPrompt, model="gpt-4"))

>>>>
1. Limited reasoning capabilities: Language models are designed to understand and generate human-like text based on patterns they have learned from large datasets. They do not possess the ability to reason or think analytically like humans do. Their responses are based on statistical associations rather than logical reasoning.

2. Lack of domain-specific knowledge: Language models may not have in-depth knowledge of specific domains or subjects, which is often required for analytical thinking and problem-solving tasks. They can only provide information that they have been trained on, and may not be able to generate accurate or relevant responses for specialized topics.

3. Inability to handle complex calculations: Language models are not designed to perform complex calculations or mathematical operations, which are often required in analytical tasks. They may struggle to provide accurate results for problems that involve numerical data or require mathematical reasoning.

4. Difficu

In [10]:

result = (600 / 2 * 30000 * 365)
print("{:.2e}".format(result))

3.28e+09


In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

## Fermi Estimation Prompt

In [ ]:

question = "How tall, in feet, is the 22nd tallest building in the world?"
# ask LLM to give its estimation chain of thought.
prompt_reason = f"""
You are an AI assistant helping people practice Fermi estimation techniques.\
A human will post a question and ask for help to estimate a quantity in the order of magnitude.\
The estimation should rely on human's common senses and knowledge about the world.

Please reason through the following steps before generate your output.
1. Identify the final quantity of interest, T, or the target quantity, that needs to be estimated.\
2. Creatively generate a list of factors that affect the target quantity, and the any relevant sub-quantities.\
3. Logically link relevant factors together to create a mathematical model.
4. Make sure the units are coherent in your model.


The question is delimited with triple backticks.
Question: ```{question}```
Reason: [INSERT REASONING]
Answer: [INSERT FINAL ANSWER]
"""
response = get_completion(prompt_reason)
print(response)

In [ ]:
question = "What is the mass of Mars in iPhone 5s?"
# ask LLM to give its estimation chain of thought.
prompt_reason = f"""
You are an AI assistant helping people practice Fermi estimation techniques.\
A human will post a question and ask for help to estimate a quantity inquired by the question in the order of magnitude.\
The estimation should rely on human's common senses and knowledge about the world.

Please reason through the following steps before generate your output.
1. Identify the final quantity of interest, T, or the target quantity, that needs to be estimated.\
2. Creatively generate a list of factors that affect the target quantity, and the any relevant sub-quantities.\
3. Logically link relevant factors together to create a mathematical model.
4. Make sure the units are coherent in your model.


The question is delimited with triple backticks.
Question: ```{question}```
Reason: [INSERT REASONING]
Answer: [INSERT FINAL ANSWER]
"""
response = get_completion(prompt_reason)
print(response)

In [ ]:
# ask LLM to give its estimation chain of thought.
prompt_wolfram = f"""

I want to infer the volumn of a cylinder with a height of 5 meter and radius of 30cm. Help me right a prompt for Wolfram Alpha web api.

"""
response1 = get_completion(prompt_wolfram,model="gpt-3.5-turbo")
print(response1)
response2 = get_completion(prompt_wolfram,model="gpt-4-0314")
print(response2)

# Formalised Estimation Q&A Procedure

In [ ]:
# Step 0: classify the question as a Fermi Question
question = "Despite what you may have learned in chemistry class, any pair of atoms will stick to each other to form a diatomic molecule. Given this fact, how many species of diatomic molecules are there?"
prompt_1 = f"""

You are an AI assistant helping people practice Fermi estimation techniques.
A curious user will ask you an estimation question after your see the delimitators ===.
A Fermi Question requires a quick, estimated answer which would be impractical to measure directly.
Your task is to classify if the user input is a valid Fermi Question or Fermi Problem.

Follow the examples to determine if the user input is a qualified Fermi Question.

User: How much does the Thames River heat up in going over the Fanshawe Dam? (Celsius degrees).
Answer: true

User: What is the mass of all the automobiles scrapped in North America this month? (kilograms)."
Answer: true

User: What's the longer river on earth?
Answer: false

User: What's the probability of tomorrow being a sunny day?
Answer: false

===
User: {question}
Answer:
"""
print("ROUND0 - CLASSIFICATION")
response1 = get_completion(prompt_1,model="gpt-3.5-turbo")
response2 = get_completion(prompt_1,model="gpt-4-0314")

# KNOWN issue:
# for this question (with more context):
# Despite what you may have learned in chemistry class, any pair of atoms will stick to each other to form a diatomic molecule. 
# Given this fact, how many species of diatomic molecules are there?

# GPT3.5 classify as true while GPT4 classify as false -> how?

In [ ]:
# Step 1: ask LLM to 'understand' the input question first.
question = "How many ice creams are consumed in London city?"
prompt_1 = f"""

You are an AI assistant helping people practice Fermi estimation techniques.\
A curious user will ask you an estimation question and test if you understand the meaning of the question. \
You will respond by paraphrase the question into a statement. \
For example, if the user ask: what is the number of piano tuners in Chicago? \
Your response will be: You'd like to estimate the number of piano tuners in the city of Chicago, the USA.
If you don't understand the question, just respond: "Sorry I don't fully understand the question. Could you please clarify?" \
If the question is not relevant to quantity estimation, respond with "Sorry, but I cannot answer that question." \
Respond with maximum 3 sentences.

User: {question}
Answer:
"""
print("ROUND1 - CLASSIFICATION")
response1 = get_completion(prompt_1,model="gpt-3.5-turbo")
print("GPT3.5: " + response1)
response2 = get_completion(prompt_1,model="gpt-4-0314")
print("GPT4: " + response2)



In [ ]:
# Step 2: once the answer is accepted, continue with information extraction
def prompt_2(original_question, model_response):
    return f"""
        A curious user asked an estimation question and you responded with your interpretation. \
        The user asked: {original_question}
        You responded: {model_response}
        Now will extract information by identifying the final quantify of interest,\
        and what factors this quantity is dependent on. Do not give numerical estimation yet.
        Respond with maximum 5 sentences.

        Answer:
        """
prompt2_1 = prompt_2(question, response1)
prompt2_2 = prompt_2(question, response2)
print("ROUND2 - CLASSIFICATION")
response2_1 = get_completion(prompt2_1,model="gpt-3.5-turbo")
print("GPT3.5: " + response2_1)
response2_2 = get_completion(prompt2_2,model="gpt-4-0314")
print("GPT4: " + response2_2)

ROUND2 - CLASSIFICATION
>>>>
GPT3.5: The final quantity of interest is the width of the green door on Mars. This quantity is dependent on several factors such as the existence of a green door on Mars, the location of the door, the method of measurement, and the accuracy of the measurement tool. Additionally, the width of the door may vary depending on its purpose and design. Without further information, it is difficult to provide a precise estimation of the width of the green door on Mars.
>>>>
GPT4: The final quantity of interest in this estimation question is the width of a green door on Mars. This quantity would depend on factors such as the purpose of the door, the size of the structure it is attached to, and the design preferences of the creators. Additionally, the width may be influenced by the materials used to construct the door and any specific requirements for the Martian environment, such as pressure differences or temperature fluctuations. Lastly, the width could also be affected by any potential Martian inhabitants' size and accessibility needs.

GPT3.5: The final quantity of interest is the proportion of Earth's water supply that is held within living organisms. This quantity is dependent on various factors such as the total amount of water on Earth, the distribution of water across different ecosystems, and the size and number of living organisms. It is also influenced by the water requirements and water retention capabilities of different species. Understanding this proportion can provide insights into the role of living organisms in the global water cycle and the impact of human activities on water resources. However, obtaining an accurate estimate of this proportion can be challenging due to the complexity and variability of ecosystems and the lack of comprehensive data.

GPT4: The final quantity of interest is the fraction of Earth's total water supply contained within all living beings. This quantity depends on factors such as the total volume of water on Earth, the biomass of all living organisms, and the average water content within these organisms. To estimate this fraction, we would need to gather data on the global water distribution, the total biomass of various life forms, and their respective water content percentages. By combining this information, we can calculate the total amount of water contained within living beings and compare it to the Earth's total water supply. This will give us an estimation of the fraction of water contained within all forms of life on our planet.

In [ ]:
# Step 3: tabulate
def prompt_3(original_question, model_response_2):
    return f"""
        A curious user asked an estimation question and you responded with your interpretation. \
        The user asked: {original_question} \
        You identified final quantity of interest and what factors or other quantities it depends on. \
        Now you will extract all relevant quantities from your response into a Markdown table: {model_response_2}. \
        For example:\
        | Quantity | value | unit |\
        |-----|-----|-----|\
        |[final quantity of interest] |[Leave this bank]|[unit based on the question]|\
        |[relevant quantity A] |[Leave this bank] |[unit based on the question] |\
        |...|...|...|\
        
        Answer with only markdown table and no conversations.
        ---
        Answer:
        """

prompt3_1 = prompt_3(question, response2_1)
prompt3_2 = prompt_3(question, response2_2)
print("ROUND3 - MAKE TABLE")
response3_1 = get_completion(prompt3_1,model="gpt-3.5-turbo")
print("GPT3.5: " + "\n" + response3_1)
response3_2 = get_completion(prompt3_2,model="gpt-4-0314")
print("GPT4: " + "\n" + response3_2)

| Quantity | value | unit |
|-----|-----|-----|
|[final quantity of interest] |[Leave this bank]|[unit based on the question]|
|[population of the city] |[Leave this bank] |[unit based on the question] |
|[weather conditions] |[Leave this bank] |[unit based on the question] |
|[availability of ice cream vendors] |[Leave this bank] |[unit based on the question] |
|[time of year] |[Leave this bank] |[unit based on the question] |
|[cultural preferences] |[Leave this bank] |[unit based on the question] |
|[age demographics] |[Leave this bank] |[unit based on the question] |
|[economic factors] |[Leave this bank] |[unit based on the question] |

GPT4:
| Quantity                          | Value | Unit                  |
|-----------------------------------|-------|-----------------------|
| Final Quantity of Interest        |       | Ice creams consumed   |
| Population of London              |       | People                |
| Percentage of Ice Cream Consumers |       | Percentage            |
| Frequency of Consumption          |       | Times per week/month  |
| Seasonal Variations               |       | Ice creams per season |
| Number of Tourists                |       | People                |
| Tourist Ice Cream Consumption     |       | Ice creams per tourist|

In [ ]:
# Step 4: From Table to JSON
import json
example_json = {
  "name": "root",
  "children": [
    {
      "name": "child1",
      "children": [
        {
          "name": "child1.1",
          "children": []
        }
      ]
    },
    {
      "name": "child2",
      "children": [
        {
          "name": "child2.1",
          "children": []
        }
      ]
    }
  ]
}
example = json.dumps(example_json)
def prompt_4(original_question, model_response_table, example=example):
    return f"""
        A curious user asked an estimation question and you responded with your interpretation. \
        The user asked: {original_question} \
        You identified final quantity of interest and what factors or other quantities it depends on. \
        Then you extract all relevant quantities into a Markdown table: \
        {model_response_table}. \
        Now you will transform the information from the table into a nested JSON object \
        with your interpretation of quantity inter-dependencies in a parent-children relation.
        For example: {example}
        Answer between the delimitator ``````, with only a JSON string and no other conversations.
        ---
        Answer:
        """
prompt4_1 = prompt_4(question, response3_1)
prompt4_2 = prompt_4(question, response3_2)
print("ROUND3 - MAKE TABLE")
response4_1 = get_completion(prompt4_1,model="gpt-3.5-turbo")
print("GPT3.5: " + "\n" + response4_1)
response4_2 = get_completion(prompt4_2,model="gpt-4-0314")
print("GPT4: " + "\n" + response4_2)

In [10]:
# extract JSON from a string
def remove_delimiters(delimiters, s):
    new_s = s
    for i in delimiters: #replace each delimiter in turn with a space
        new_s = new_s.replace(i, ' ')
    return ' '.join(new_s.split())

json_1 = json.loads(remove_delimiters("`", response4_1)) 
json_2 = json.loads(remove_delimiters("`", response4_2)) 
print(json_1)
print(json_2)

NameError: name 'response4_1' is not defined

# Combining Step 1-4

In [3]:
# Step 1: clarification - check whether LLM 'understands' the input question.

## reflection => what if we replace this step to "capture user's intention"?
def clarify(query):
    return f"""\
A curious user will ask you an estimation question and test if you understand what the question means.
You will respond by paraphrase the question into a statement.
For example, if the user ask: what is the number of piano tuners in Chicago?
Your response will be: You'd like to estimate the number of piano tuners in the city of Chicago, the USA.
If you don't understand the question, respond: "Sorry I don't fully understand the question. Could you please clarify?"
If the question is not relevant to quantity estimation, respond with "Sorry, but I cannot answer that question."
Respond with maximum 3 sentences.

User: {query}
Answer:
"""

# Step 2: information extraction - if user accept model interpretation start extract information
def extract(original_question, model_response):
    return f"""\
A curious user asked an estimation question and you responded with your interpretation.
The user asked: {original_question}
You responded: {model_response}
Now will extract information by identifying the final quantify of interest,
and what factors this quantity is dependent on. Do not give numerical estimation yet.
Respond with maximum 5 sentences.

Answer:
"""

# step 3: 
table = f"""\
| Quantity | value | unit |
|-----|-----|-----|
|[final quantity of interest] |[Leave this bank]|[unit based on the question]|
|[relevant quantity A] |[Leave this bank] |[unit based on the question] |
|...|...|...|
"""

def tabulate(original_question, model_response_2, example=table):
    return f"""\
A curious user asked an estimation question and you responded with your interpretation. 
The user asked: {original_question}
You identified final quantity of interest, and what factors or other quantities it depends on.
Now you will translate all relevant quantities from your response into a Markdown table: 
{model_response_2}.
For example:
{example}

Answer with only markdown table and no conversations.
---
Answer:
"""

# Step 4: data transformation (convert table to json - can we hard-code this?)
example_json = {
  "name": "quantity of interest",
  "children": [
    {
      "name": "sub-quantity 1",
      "children": [
        {
          "name": "sub-quantity 1.1",
          "children": []
        }
      ]
    },
    {
      "name": "sub-quantity 2",
      "children": [
        {
          "name": "sub-quantity 2.1",
          "children": []
        }
      ]
    }
  ]
}
zero_shot_example = json.dumps(example_json)
def convertToJSON(original_question, model_response_table, example=zero_shot_example):
    return f"""\
A curious user asked an estimation question and you responded with your interpretation.
The user asked: {original_question}
You identified final quantity of interest and what factors or other quantities it depends on.
Then you extracted all relevant quantities into a Markdown table:
{model_response_table}.
Now you will transform the information from the table into a nested JSON object
with your interpretation of quantity inter-dependencies in a parent-children relation.
For example: 
{example}
Answer between the delimitator ``````, with only a JSON string and no other conversations.
---
Answer:```[your answer]```
"""

# output validation
# extract JSON from a string
def remove_delimiters(delimiters, s):
    new_s = s
    for i in delimiters: #replace each delimiter in turn with a space
        new_s = new_s.replace(i, ' ')
    return ' '.join(new_s.split())

In [4]:
##
# Inference
question = "How many characters are there in this sentence??"

for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)

    print("- User query: " + question)
    response1 = get_completion(clarify(question))

    print("- Extracted information: ")
    response2 = get_completion(extract(question, response1))

    print("- Format to Table:")
    response3 = get_completion(tabulate(question, response2))

    print("- Format to JSON:")
    response4 = get_completion(convertToJSON(question, response3))
    # note sometimes the response will return with ``````, so we can use remove_delimiters function to filter out,
    # or use a "json extractor" function to extract the first json object from a returned message (risk of prompt injection)
    try:
        json_1 = json.loads(remove_delimiters("`", response4))
        print("Validated JSON:")
        print(json_1)
    except Exception as e:
        print(e)





gpt-3.5-turbo-0301
- User query: How many characters are there in this sentence??
>>>>
You're asking for an estimate of the number of characters in the sentence you just wrote. It appears to be 38 characters long, including spaces and punctuation.
- Extracted information: 
>>>>
The final quantity of interest is the number of characters in the sentence. This quantity is dependent on the length of the sentence, including spaces and punctuation. The sentence may also contain special characters or emojis, which would affect the final count. The font and size of the text may also impact the character count. Finally, the language used in the sentence may have different character counts for certain letters or symbols.
- Format to Table:
>>>>
| Quantity | value | unit |
|-----|-----|-----|
|Final quantity of interest |Number of characters |Count |
|Length of sentence |Varies |Count |
|Special characters/emojis |Varies |Count |
|Font and size of text |Varies |N/A |
|Language used |Varies |N/A |

In [ ]:
# GPT4
print("- User query: " + question)
response1 = get_completion(clarify(question), model="gpt-4")

print("- Extracted information: ")
response2 = get_completion(extract(question, response1), model="gpt-4")

print("- Format to Table:")
response3 = get_completion(tabulate(question, response2), model="gpt-4")

print("- Format to JSON:")
response4 = get_completion(convertToJSON(question, response3), model="gpt-4")
# note sometimes the response will return with ``````, so we can use remove_delimiters function to filter out,
# or use a "json extractor" function to extract the first json object from a returned message (risk of prompt injection)
json_1 = json.loads(remove_delimiters("`", response4))
print(json_1)

In [ ]:
# let's try to count the number of tokens for each query
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(tabulate(question, response2), "cl100k_base")

# count the number of tokens from the last response



In [7]:
# check assumption


def checkAssumption(statement):
    template = """Here is a statement:
{statement}
Make a bullet point list of the assumptions the user made when producing the given statement.\n\n"""
    return template

statement = "What's the size of the green door on the Moon?"

for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)
    print("- Check assumption: " + statement)
    response = get_completion(checkAssumption(statement), model)



gpt-3.5-turbo-0301
- Check assumption: What's the size of the green door on the Moon?
>>>>
As an AI language model, I cannot provide assumptions made by the user without any context or information about the statement. Please provide more details or context for me to assist you better.
gpt-4-0314
- Check assumption: What's the size of the green door on the Moon?
>>>>
- The user assumes that the statement is clear and understandable.
- The user assumes that the statement is based on accurate information or personal beliefs.
- The user assumes that the statement is relevant to the context or topic being discussed.
- The user assumes that the statement will be received and interpreted as intended.
- The user assumes that the statement will provoke thought or elicit a response from the audience.
- The user assumes that the statement is appropriate for the audience or platform it is being shared on.


In [ ]:
table = f"""\
        A curious user asked an estimation question and you responded with your interpretation.
        The user asked: {question}
        You identified final quantity of interest and what factors or other quantities it depends on.
        Then you extract all relevant quantities into a Markdown table:
        {response2_1}.
        Now you will transform the information from the table into a nested JSON object
        with your interpretation of quantity inter-dependencies in a parent-children relation.
        For example: {example}
        Answer between the delimitator ``````, with only a JSON string and no other conversations.
        ---
        Answer:
        """

print(table)

table2 = f"""\
| Quantity | value | unit |
|-----|-----|-----|
|[final quantity of interest] |[Leave this bank]|[unit based on the question]|
|[relevant quantity A] |[Leave this bank] |[unit based on the question] |
|...|...|...|
"""
print(table2)

In [13]:
# check assumption


def determineFactor(user_query):
    template = f"""\
Please assist me in identifying the key variables or factors that would influence the estimation of the following question: {user_query}. \
Please list them in bullet points.\n
Then try to form a mathematical model."""
    return template

user_query = "What's the weight of Lithum in obsolete batteries from electric cars produced in 2020?"

for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)
    print("- Check assumption: " + user_query)
    response = get_completion(determineFactor(user_query), model)

gpt-3.5-turbo-0301
- Check assumption: What's the weight of Lithum in obsolete batteries from electric cars produced in 2020?
>>>>
Key variables/factors:
- Number of electric cars produced in 2020
- Average weight of lithium in a single battery
- Average number of batteries per electric car
- Percentage of obsolete batteries from electric cars produced in 2020
- Efficiency of the recycling process for lithium in batteries

Mathematical model:
Weight of Lithium in Obsolete Batteries = (Number of Electric Cars Produced in 2020) x (Average Weight of Lithium in a Single Battery) x (Average Number of Batteries per Electric Car) x (Percentage of Obsolete Batteries from Electric Cars Produced in 2020) x (Efficiency of the Recycling Process for Lithium in Batteries)
gpt-4-0314
- Check assumption: What's the weight of Lithum in obsolete batteries from electric cars produced in 2020?
>>>>
Key variables or factors:

- Number of electric cars produced in 2020
- Percentage of electric cars using li

In [6]:
prompt = f"""\
Solve the following equation, show your working steps, and then finally return the result in a dictionary where key is the variable, and the value is the value after the flag token.
Question: [a + b - 3c + 1 = 0; -b - 2 = 0, c + 1 = 0]
Solution:
Result:
"""
for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)
    response = get_completion(prompt, model)

gpt-3.5-turbo-0301
>>>>
First, we can use the second equation to solve for b:
-b - 2 = 0
-b = 2
b = -2

Next, we can use the third equation to solve for c:
c + 1 = 0
c = -1

Substituting these values into the first equation, we get:
a - 6 = 0
a = 6

Therefore, the solution is:
{'a': 6, 'b': -2, 'c': -1}
gpt-4-0314
>>>>
First, let's solve the second and third equations for b and c, respectively.

Equation 2: -b - 2 = 0
Add 2 to both sides:
-b = 2
Multiply both sides by -1:
b = -2

Equation 3: c + 1 = 0
Subtract 1 from both sides:
c = -1

Now, let's substitute the values of b and c into the first equation.

Equation 1: a + b - 3c + 1 = 0
Substitute b = -2 and c = -1:
a + (-2) - 3(-1) + 1 = 0

Simplify:
a - 2 + 3 + 1 = 0
a + 2 = 0

Subtract 2 from both sides:
a = -2

Now, we can write the result in a dictionary:

Result: {'a': -2, 'b': -2, 'c': -1}


In [8]:
prompt = f"""\
A curious learner is asking you a question:
How many ice creams are sold in Brighton, UK each year?
You know the answer is in the order of magnitude, and you will now tell the learner, step by step, how they can work out the answer themselves. End your response with 10^(n), where n is your estimated answer.
"""
for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)
    response = get_completion(prompt, model)

gpt-3.5-turbo-0301
>>>>
To estimate the number of ice creams sold in Brighton, UK each year, we can follow these steps:

1. Find the population of Brighton, UK. According to the latest census data, the population of Brighton is around 290,000.

2. Estimate the number of ice creams consumed per person per year. This can vary widely depending on factors such as age, income, and weather. However, a reasonable estimate could be around 20-30 ice creams per person per year.

3. Multiply the population by the estimated number of ice creams consumed per person per year. Using the midpoint of the estimate (25 ice creams per person per year), we get:

290,000 x 25 = 7,250,000

So, based on these estimates, we can say that around 7-8 million ice creams are sold in Brighton, UK each year. This can be expressed in order of magnitude as 10^7.
gpt-4-0314
>>>>
To estimate the number of ice creams sold in Brighton, UK each year, we can follow these steps:

1. Estimate the population of Brighton: Accord

# Experiment with Wolfram Alpha

In [43]:
def query_wolfram_alpha(query):
    app_id = os.getenv('chatWolf')
    client = wolframalpha.Client(app_id)
    res = client.query(query)

    try:
        return next(res.results).text
    except StopIteration:
        return "No results found."

# Example usage:
test_quantity_list = ["lenth", "density", "mass", "time", "power",
                       "energy", "force", "pressure", "velocity", "acceleration",
                         "area", "volume", "temperature", "charge", "voltage", "current",
                           "resistance", "magnetic field", "inductance", "frequency", "angle",
                             "angular velocity", "angular acceleration", "momentum", "angular momentum",
                               "torque", "moment of inertia", "impulse", "work", "power", "energy", "density",
                                 "specific heat", "heat capacity", "thermal conductivity", "thermal expansion",
                                   "viscosity", "surface tension", "electric field", "electric flux", "electric potential",
                                     "electric resistance", "electric resistivity", "electric conductance", "electric conductivity",
                                       "electric capacitance", "electric inductance"] 
                                    
safe_quantity_dict = {}
unsafe_cases = []
# filter out unsafe cases
for i in range(len(test_quantity_list)):
    quantity = test_quantity_list[i]
    query = f"""dimensions of {quantity}"""
    result = query_wolfram_alpha(query)
    if (result == "No results found."):
        print(f"Quantity {quantity} found no result.")
        # check if quanity is in unsafe_cases
        if (quantity not in unsafe_cases):
            unsafe_cases.append(quantity)
    # update safe_quantity_dict, key is quantity, value is result
    else:
        print("Updaintg safe_quantity_dict:" + quantity + "=" + result )
        safe_quantity_dict[quantity] = result




Updaintg safe_quantity_dict:lenth=[length]^
Quantity density found no result.
Updaintg safe_quantity_dict:mass=[mass]^
Updaintg safe_quantity_dict:time=[time]^
Updaintg safe_quantity_dict:power=[length]^2 [mass]^ [time]^(-3)
Updaintg safe_quantity_dict:energy=[length]^2 [mass]^ [time]^(-2)
Updaintg safe_quantity_dict:force=[length]^ [mass]^ [time]^(-2)
Updaintg safe_quantity_dict:pressure=[length]^(-1) [mass]^ [time]^(-2)
Updaintg safe_quantity_dict:velocity=[length]^ [time]^(-1)
Updaintg safe_quantity_dict:acceleration=[length]^ [time]^(-2)
Updaintg safe_quantity_dict:area=[length]^2
Updaintg safe_quantity_dict:volume=[length]^3
Updaintg safe_quantity_dict:temperature=[temperature]^
Quantity charge found no result.
Updaintg safe_quantity_dict:voltage=[electric current]^(-1) [length]^2 [mass]^ [time]^(-3)
Quantity current found no result.
Quantity resistance found no result.
Quantity magnetic field found no result.
Quantity inductance found no result.
Updaintg safe_quantity_dict:freque

In [47]:
print(unsafe_cases)
print(safe_quantity_dict)

# save safe_quantity_dict to json file
with open('safe_quantity_dict.json', 'w') as fp:
    json.dump(safe_quantity_dict, fp, indent=2)


['density', 'charge', 'current', 'resistance', 'magnetic field', 'inductance', 'angle', 'specific heat', 'thermal expansion', 'viscosity', 'electric field', 'electric inductance']
{'lenth': '[length]^', 'mass': '[mass]^', 'time': '[time]^', 'power': '[length]^2 [mass]^ [time]^(-3)', 'energy': '[length]^2 [mass]^ [time]^(-2)', 'force': '[length]^ [mass]^ [time]^(-2)', 'pressure': '[length]^(-1) [mass]^ [time]^(-2)', 'velocity': '[length]^ [time]^(-1)', 'acceleration': '[length]^ [time]^(-2)', 'area': '[length]^2', 'volume': '[length]^3', 'temperature': '[temperature]^', 'voltage': '[electric current]^(-1) [length]^2 [mass]^ [time]^(-3)', 'frequency': '[time]^(-1)', 'angular velocity': '[angle]^ [time]^(-1)', 'angular acceleration': '[angle]^ [time]^(-2)', 'momentum': '[length]^ [mass]^ [time]^(-1)', 'angular momentum': '[length]^2 [mass]^ [time]^(-1)', 'torque': '[length]^2 [mass]^ [time]^(-2)', 'moment of inertia': '[length]^2 [mass]^', 'impulse': '[length]^ [mass]^ [time]^(-1)', 'work